#### Make new features
1. Focus on featured_importance and domain knowledge.
2. Recall top 20 importance from Random Forest:
['EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH', 'EXT_SOURCE_1', 'DAYS_ID_PUBLISH', 'DAYS_REGISTRATION', 'df_avg_bureau_full_DAYS_CREDIT', 'df_avg_bureau_full_DAYS_CREDIT_ENDDATE', 'AMT_PAYMENT_df_avg_install', 'AMT_ANNUITY', 'DAYS_EMPLOYED', 'df_avg_bureau_full_DAYS_CREDIT_UPDATE', 'df_avg_pos_cash_CNT_INSTALMENT_FUTURE', 'AMT_INSTALMENT_df_avg_install', 'DAYS_LAST_PHONE_CHANGE', 'AMT_CREDIT', 'DAYS_ENTRY_PAYMENT_df_avg_install', 'DAYS_INSTALMENT_df_avg_install', 'df_avg_previous_app_DAYS_FIRST_DUE', 'df_avg_previous_app_DAYS_DECISION', 'df_avg_previous_app_HOUR_APPR_PROCESS_START', 'df_avg_previous_app_AMT_ANNUITY', 'df_avg_previous_app_AMT_CREDIT', 'df_avg_previous_app_AMT_GOODS_PRICE', 'df_avg_previous_app_AMT_APPLICATION', 'df_avg_previous_app_SELLERPLACE_AREA', 'REGION_POPULATION_RELATIVE', 'df_avg_previous_app_DAYS_LAST_DUE_1ST_VERSION', 'df_avg_bureau_full_AMT_CREDIT_SUM', 'AMT_INCOME_TOTAL']


In [1]:
## IMPORTS ##

# numpy and pandas for data manipulation
import numpy as np
import pandas as pd 

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

# File system manangement
import os

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

# matplotlib for plotting
import matplotlib.pyplot as plt

# garbage collector
import gc

In [2]:
# Load data fcn
def load_credit_data(data_path):
    csv_path = os.path.join("data", data_path)
    return pd.read_csv(csv_path)

In [3]:
# Load training data
training_df = load_credit_data ("application_train.csv")

print (training_df.shape)
training_df.head()

(307511, 122)


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
# Load test data
testing_df = load_credit_data ("application_test.csv")

print (testing_df.shape)
testing_df.head()

(48744, 121)


,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
4,100038,Cash loans,M,Y,N,1,180000.0,625500.0,32067.0,625500.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
tr_features_df = training_df[['SK_ID_CURR']]
te_features_df = testing_df[['SK_ID_CURR']]

In [6]:
# New application features from feature importance and domain knowledge
    
def add_new_features (df_new, existing_df):
    df_new['APP_CREDIT_TO_ANNUITY_RATIO'] = existing_df['AMT_CREDIT'] / existing_df['AMT_ANNUITY']
    df_new['APP_CREDIT_TO_GOODS_RATIO'] = existing_df['AMT_CREDIT'] / existing_df['AMT_GOODS_PRICE']
    df_new['APP_INCOME_PER_CHLD'] = existing_df['AMT_INCOME_TOTAL'] / (1 + existing_df['CNT_CHILDREN'])
    df_new['APP_CREDIT_TO_INCOME_RATIO'] = existing_df['AMT_CREDIT'] / existing_df['AMT_INCOME_TOTAL']
    df_new['APP_ANNUITY_TO_INCOME_RATIO'] = existing_df['AMT_ANNUITY'] / (1 + existing_df['AMT_INCOME_TOTAL'])
    df_new['APP_SOURCES_PROD'] = existing_df['EXT_SOURCE_1'] * existing_df['EXT_SOURCE_2'] * existing_df['EXT_SOURCE_3']
    df_new['APP_EXT_SOURCES_MEAN'] = existing_df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)
    df_new['APP_EXT_SCORES_STD'] = existing_df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].std(axis=1)
    df_new['APP_CAR_TO_BIRTH_RATIO'] = existing_df['OWN_CAR_AGE'] / existing_df['DAYS_BIRTH']
    df_new['APP_CAR_TO_EMPLOY_RATIO'] = existing_df['OWN_CAR_AGE'] / existing_df['DAYS_EMPLOYED']
    df_new['APP_PHONE_TO_BIRTH_RATIO'] = existing_df['DAYS_LAST_PHONE_CHANGE'] / existing_df['DAYS_BIRTH']
    df_new['APP_PHONE_TO_EMPLOY_RATIO'] = existing_df['DAYS_LAST_PHONE_CHANGE'] / existing_df['DAYS_EMPLOYED']
    df_new['APP_DAYS_PUBLISH_TO_BIRTH_RATIO'] = existing_df['DAYS_ID_PUBLISH'] / existing_df['DAYS_BIRTH']
    df_new['APP_DAYS_PUBLISH_TO_EMPLOY_RATIO'] = existing_df['DAYS_ID_PUBLISH'] / existing_df['DAYS_EMPLOYED']
    df_new['APP_DAYS_REGISTRATION_TO_BIRTH_RATIO'] = existing_df['DAYS_REGISTRATION'] / existing_df['DAYS_BIRTH']
    df_new['APP_DAYS_REGISTRATION_TO_EMPLOY_RATIO'] = existing_df['DAYS_REGISTRATION'] / existing_df['DAYS_EMPLOYED']
    df_new['APP_REGION_DATA_PROD'] = existing_df['REGION_POPULATION_RELATIVE'] * existing_df['REGION_RATING_CLIENT'] * existing_df['REGION_RATING_CLIENT_W_CITY']
    df_new['APP_REGION_DATA_MEAN'] = existing_df[['REGION_POPULATION_RELATIVE', 'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY']].mean(axis=1)
    df_new['APP_REGION_DATA_STD'] = existing_df[['REGION_POPULATION_RELATIVE', 'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY']].std(axis=1)
   
    return df_new

In [7]:
tr_features_df = add_new_features (tr_features_df, training_df)
tr_features_df.fillna(tr_features_df.median(), inplace=True)
tr_features_df.head()

,SK_ID_CURR,APP_CREDIT_TO_ANNUITY_RATIO,APP_CREDIT_TO_GOODS_RATIO,APP_INCOME_PER_CHLD,APP_CREDIT_TO_INCOME_RATIO,APP_ANNUITY_TO_INCOME_RATIO,APP_SOURCES_PROD,APP_EXT_SOURCES_MEAN,APP_EXT_SCORES_STD,APP_CAR_TO_BIRTH_RATIO,APP_CAR_TO_EMPLOY_RATIO,APP_PHONE_TO_BIRTH_RATIO,APP_PHONE_TO_EMPLOY_RATIO,APP_DAYS_PUBLISH_TO_BIRTH_RATIO,APP_DAYS_PUBLISH_TO_EMPLOY_RATIO,APP_DAYS_REGISTRATION_TO_BIRTH_RATIO,APP_DAYS_REGISTRATION_TO_EMPLOY_RATIO,APP_REGION_DATA_PROD,APP_REGION_DATA_MEAN,APP_REGION_DATA_STD
0,100002,16.461104,1.158397,202500.0,2.007889,0.121977,0.003043,0.161787,0.092026,-0.000627,-0.004357,0.119860,1.780220,0.224078,3.328100,0.385583,5.726845,0.075204,1.339600,1.143846
1,100003,36.234085,1.145199,270000.0,4.790750,0.132216,0.119932,0.466757,0.219895,-0.000627,-0.004357,0.049389,0.696970,0.017358,0.244949,0.070743,0.998316,0.003541,0.667847,0.575306
2,100004,20.000000,1.000000,67500.0,2.000000,0.099999,0.119932,0.642739,0.122792,-0.001365,-0.115556,0.042791,3.622222,0.132889,11.248889,0.223669,18.933333,0.040128,1.336677,1.148909
3,100006,10.532818,1.052803,135000.0,2.316167,0.219898,0.119932,0.650442,0.136021,-0.000627,-0.004357,0.032465,0.203027,0.128229,0.801909,0.517390,3.235604,0.032076,1.336006,1.150071
4,100007,23.461618,1.000000,121500.0,4.222222,0.179961,0.119932,0.322738,0.136021,-0.000627,-0.004357,0.055489,0.364055,0.173490,1.138249,0.216285,1.419026,0.114652,1.342888,1.138152


In [8]:
te_features_df = add_new_features (te_features_df, testing_df)
te_features_df.fillna(te_features_df.median(), inplace=True)
te_features_df.head()

,SK_ID_CURR,APP_CREDIT_TO_ANNUITY_RATIO,APP_CREDIT_TO_GOODS_RATIO,APP_INCOME_PER_CHLD,APP_CREDIT_TO_INCOME_RATIO,APP_ANNUITY_TO_INCOME_RATIO,APP_SOURCES_PROD,APP_EXT_SOURCES_MEAN,APP_EXT_SCORES_STD,APP_CAR_TO_BIRTH_RATIO,APP_CAR_TO_EMPLOY_RATIO,APP_PHONE_TO_BIRTH_RATIO,APP_PHONE_TO_EMPLOY_RATIO,APP_DAYS_PUBLISH_TO_BIRTH_RATIO,APP_DAYS_PUBLISH_TO_EMPLOY_RATIO,APP_DAYS_REGISTRATION_TO_BIRTH_RATIO,APP_DAYS_REGISTRATION_TO_EMPLOY_RATIO,APP_REGION_DATA_PROD,APP_REGION_DATA_MEAN,APP_REGION_DATA_STD
0,100001,27.664697,1.2640,135000.0,4.213333,0.152299,0.094803,0.567263,0.353601,-0.000624,-0.004000,0.090432,0.747102,0.042202,0.348647,0.268697,2.219837,0.075400,1.339617,1.143817
1,100005,12.824870,1.2376,99000.0,2.250182,0.175453,0.071345,0.429869,0.136694,-0.000624,-0.004000,-0.000000,-0.000000,0.089847,0.363168,0.504761,2.040277,0.143168,1.345264,1.134036
2,100013,9.505482,1.0528,202500.0,3.275378,0.344576,0.119686,0.655389,0.062788,-0.000250,-0.001122,0.042719,0.192014,0.174818,0.785778,0.108544,0.487887,0.076404,1.339700,1.143673
3,100028,32.130726,1.0000,105000.0,5.000000,0.155614,0.164177,0.549372,0.055432,-0.000624,-0.004000,0.129150,0.967310,0.301088,2.255091,0.143102,1.071811,0.105568,1.342131,1.139463
4,100038,19.506034,1.0000,90000.0,3.475000,0.178149,0.119686,0.313916,0.158068,-0.001227,-0.007303,0.062960,0.374715,0.326840,1.945230,0.306748,1.825650,0.040128,1.336677,1.148909


In [9]:
# Bureau data
bureau_df = load_credit_data ("bureau.csv")
bureau_bal_df = load_credit_data ("bureau_balance.csv")

# Previously, I got the sum. Now, try other aggregates.

# Aggregates
aggregations = {'MONTHS_BALANCE': ['max', 'min', 'size'],}

bureau_bal_df_avg = bureau_bal_df.groupby('SK_ID_BUREAU').agg(aggregations)
bureau_bal_df_avg.head()

MONTHS_BALANCE         
                        max min size
SK_ID_BUREAU                        
5001709                   0 -96   97
5001710                   0 -82   83
5001711                   0  -3    4
5001712                   0 -18   19
5001713                   0 -21   22

In [10]:
# Rename labels
bureau_bal_df_avg.columns = pd.Index(['PREVIOUS_' + e[0] + "_" + e[1].upper() for e in bureau_bal_df_avg.columns.tolist()])

bureau_bal_df_avg.head()

,PREVIOUS_MONTHS_BALANCE_MAX,PREVIOUS_MONTHS_BALANCE_MIN,PREVIOUS_MONTHS_BALANCE_SIZE
SK_ID_BUREAU,,,
5001709,0,-96,97
5001710,0,-82,83
5001711,0,-3,4
5001712,0,-18,19
5001713,0,-21,22


In [11]:
# Merge by SK_ID_BUREAU
bureau_full_df = bureau_df.merge(right=bureau_bal_df_avg.reset_index(), on = 'SK_ID_BUREAU', how = 'left')
print (bureau_df.shape)
print (bureau_full_df.shape)

(1716428, 17)
(1716428, 20)


In [12]:
bureau_full_df.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,PREVIOUS_MONTHS_BALANCE_MAX,PREVIOUS_MONTHS_BALANCE_MIN,PREVIOUS_MONTHS_BALANCE_SIZE
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN,NaN,NaN,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN,NaN,NaN,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN,NaN,NaN,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN,NaN,NaN,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN,NaN,NaN,NaN


In [13]:
# Deal with NULLs from the merges - if no info exists, just assume 0.
bureau_full_df.fillna(0, inplace=True)

In [14]:
# Get counts of number of bureau balances
bureau_full_df_prevs = bureau_full_df[['SK_ID_CURR', 'SK_ID_BUREAU']].groupby('SK_ID_CURR').count()
bureau_full_df['SK_ID_BUREAU'] = bureau_full_df['SK_ID_CURR'].map(bureau_full_df_prevs['SK_ID_BUREAU'])

# Create new df 
new_bureau_df = bureau_full_df[['SK_ID_CURR', 'SK_ID_BUREAU']]

# Add on features (and some aggregation of existing columns)
# TOO many 0/NULLs i.e. over 60% - new_bureau_df['CREDIT_MAX_OVERDUE_TO_LIMIT_RATIO'] =  bureau_full_df['AMT_CREDIT_MAX_OVERDUE'] / bureau_full_df['AMT_CREDIT_SUM_LIMIT']
new_bureau_df['CNT_CREDIT_PROLONG'] =  bureau_full_df['CNT_CREDIT_PROLONG']
new_bureau_df['PREVIOUS_MONTHS_BALANCE_MAX'] =  bureau_full_df['PREVIOUS_MONTHS_BALANCE_MAX']
new_bureau_df['PREVIOUS_MONTHS_BALANCE_MIN'] =  bureau_full_df['PREVIOUS_MONTHS_BALANCE_MIN'] 
new_bureau_df['PREVIOUS_MONTHS_BALANCE_SIZE'] =  bureau_full_df['PREVIOUS_MONTHS_BALANCE_SIZE']

# Get more aggregates for feature important features
new_bureau_df['DAYS_CREDIT'] =  bureau_full_df['DAYS_CREDIT']
new_bureau_df['DAYS_CREDIT_UPDATE'] =  bureau_full_df['DAYS_CREDIT_UPDATE']
new_bureau_df['DAYS_CREDIT_ENDDATE'] =  bureau_full_df['DAYS_CREDIT_ENDDATE']

# Drop BUREAU_ID as would already exist in base file - could leave in if needed
new_bureau_df = new_bureau_df.drop ('SK_ID_BUREAU', axis=1)

# Fill before aggregate
new_bureau_df.fillna(new_bureau_df.median(), inplace=True)

# Aggregates
aggregations = {
        #'CREDIT_MAX_OVERDUE_TO_LIMIT_RATIO': ['max', 'min', 'mean'],
        'CNT_CREDIT_PROLONG': ['max', 'min', 'sum'],
        'PREVIOUS_MONTHS_BALANCE_MAX': ['max'],
        'PREVIOUS_MONTHS_BALANCE_MIN': ['min', ],
        'PREVIOUS_MONTHS_BALANCE_SIZE': ['max', 'min', 'sum'],
        'DAYS_CREDIT': ['max', 'min', 'median', 'var'],
        'DAYS_CREDIT_UPDATE': ['max', 'min', 'median'],
        'DAYS_CREDIT_ENDDATE': ['max', 'min', 'median'],
}

# Aggregate - joined later to t*_features_df
bureau_full_df_avg  = new_bureau_df.groupby('SK_ID_CURR').agg(aggregations)

bureau_full_df_avg.head()

CNT_CREDIT_PROLONG         PREVIOUS_MONTHS_BALANCE_MAX  \
                          max min sum                         max   
SK_ID_CURR                                                          
100001                      0   0   0                         0.0   
100002                      0   0   0                         0.0   
100003                      0   0   0                         0.0   
100004                      0   0   0                         0.0   
100005                      0   0   0                         0.0   

           PREVIOUS_MONTHS_BALANCE_MIN PREVIOUS_MONTHS_BALANCE_SIZE       \
                                   min                          max  min   
SK_ID_CURR                                                                 
100001                           -51.0                         52.0  2.0   
100002                           -47.0                         22.0  4.0   
100003                             0.0                          0.0  0.0   
100004                             0.0                          0.0  0.0   
100005                           -12.0                         13.0  3.0   

                  DAYS_CREDIT                              DAYS_CREDIT_UPDATE  \
              sum         max   min  median            var                max   
SK_ID_CURR                                                                      
100001      172.0         -49 -1572  -857.0  240043.666667                 -6   
100002      110.0        -103 -1437 -1042.5  186150.000000                 -7   
100003        0.0        -606 -2586 -1205.5  827783.583333                -43   
100004        0.0        -408 -1326  -867.0  421362.000000               -382   
100005       21.0         -62  -373  -137.0   26340.333333                -11   

                        DAYS_CREDIT_ENDDATE                 
             min median                 max     min median  
SK_ID_CURR                                                  
100001      -155 -155.0              1778.0 -1329.0 -179.0  
100002     -1185 -402.5               780.0 -1072.0    0.0  
100003     -2131 -545.0              1216.0 -2434.0 -480.0  
100004      -682 -532.0              -382.0  -595.0 -488.5  
100005      -121  -31.0              1324.0  -128.0  122.0

In [15]:
# Rename labels
bureau_full_df_avg.columns = pd.Index(['BUREAU_' + e[0] + "_" + e[1].upper() for e in bureau_full_df_avg.columns.tolist()])

# Replace inf - not sure what this means
bureau_full_df_avg.replace([np.inf, -np.inf], 0)

bureau_full_df_avg.head()

,BUREAU_CNT_CREDIT_PROLONG_MAX,BUREAU_CNT_CREDIT_PROLONG_MIN,BUREAU_CNT_CREDIT_PROLONG_SUM,BUREAU_PREVIOUS_MONTHS_BALANCE_MAX_MAX,BUREAU_PREVIOUS_MONTHS_BALANCE_MIN_MIN,BUREAU_PREVIOUS_MONTHS_BALANCE_SIZE_MAX,BUREAU_PREVIOUS_MONTHS_BALANCE_SIZE_MIN,BUREAU_PREVIOUS_MONTHS_BALANCE_SIZE_SUM,BUREAU_DAYS_CREDIT_MAX,BUREAU_DAYS_CREDIT_MIN,BUREAU_DAYS_CREDIT_MEDIAN,BUREAU_DAYS_CREDIT_VAR,BUREAU_DAYS_CREDIT_UPDATE_MAX,BUREAU_DAYS_CREDIT_UPDATE_MIN,BUREAU_DAYS_CREDIT_UPDATE_MEDIAN,BUREAU_DAYS_CREDIT_ENDDATE_MAX,BUREAU_DAYS_CREDIT_ENDDATE_MIN,BUREAU_DAYS_CREDIT_ENDDATE_MEDIAN
SK_ID_CURR,,,,,,,,,,,,,,,,,,
100001,0,0,0,0.0,-51.0,52.0,2.0,172.0,-49,-1572,-857.0,240043.666667,-6,-155,-155.0,1778.0,-1329.0,-179.0
100002,0,0,0,0.0,-47.0,22.0,4.0,110.0,-103,-1437,-1042.5,186150.000000,-7,-1185,-402.5,780.0,-1072.0,0.0
100003,0,0,0,0.0,0.0,0.0,0.0,0.0,-606,-2586,-1205.5,827783.583333,-43,-2131,-545.0,1216.0,-2434.0,-480.0
100004,0,0,0,0.0,0.0,0.0,0.0,0.0,-408,-1326,-867.0,421362.000000,-382,-682,-532.0,-382.0,-595.0,-488.5
100005,0,0,0,0.0,-12.0,13.0,3.0,21.0,-62,-373,-137.0,26340.333333,-11,-121,-31.0,1324.0,-128.0,122.0


In [16]:
# Previous Applications

# Load  data
previous_app_df = load_credit_data ("previous_application.csv")

# Get counts of number previous applications
previous_app_df_prevs = previous_app_df[['SK_ID_CURR', 'SK_ID_PREV']].groupby('SK_ID_CURR').count()
previous_app_df['SK_ID_PREV'] = previous_app_df['SK_ID_CURR'].map(previous_app_df_prevs['SK_ID_PREV'])

# Create new df 
new_prev_app_df = previous_app_df[['SK_ID_CURR', 'SK_ID_PREV']]

# Add on features
new_prev_app_df['PAYMENT_TO_CREDIT_RATIO'] =  previous_app_df['AMT_DOWN_PAYMENT'] / previous_app_df['AMT_CREDIT']
new_prev_app_df['CREDIT_TO_APPLICATION_RATIO'] = previous_app_df ['AMT_CREDIT'] / previous_app_df['AMT_APPLICATION']
new_prev_app_df['CREDIT_TO_ANNUITY_RATIO'] = previous_app_df['AMT_CREDIT'] / previous_app_df['AMT_ANNUITY']

# Get more aggregates for feature important features
new_prev_app_df['DAYS_FIRST_DUE'] =  previous_app_df['DAYS_FIRST_DUE']
new_prev_app_df['DAYS_DECISION'] =  previous_app_df['DAYS_DECISION']
new_prev_app_df['HOUR_APPR_PROCESS_START'] =  previous_app_df['HOUR_APPR_PROCESS_START']
new_prev_app_df['AMT_ANNUITY'] =  previous_app_df['AMT_ANNUITY']
new_prev_app_df['AMT_CREDIT'] =  previous_app_df['AMT_CREDIT']
new_prev_app_df['AMT_GOODS_PRICE'] =  previous_app_df['AMT_GOODS_PRICE']
new_prev_app_df['AMT_APPLICATION'] =  previous_app_df['AMT_APPLICATION']
new_prev_app_df['SELLERPLACE_AREA'] =  previous_app_df['SELLERPLACE_AREA']
new_prev_app_df['DAYS_LAST_DUE_1ST_VERSION'] =  previous_app_df['DAYS_LAST_DUE_1ST_VERSION']

# Drop PREV as would already exist in base file - could leave in if needed
new_prev_app_df = new_prev_app_df.drop ('SK_ID_PREV', axis=1)

# Fill before aggregate
new_prev_app_df.fillna(new_prev_app_df.median(), inplace=True)

# Aggregates
aggregations = {
        'PAYMENT_TO_CREDIT_RATIO': ['max', 'min', 'mean'],
        'CREDIT_TO_APPLICATION_RATIO': ['max', 'min', 'mean'],
        'CREDIT_TO_ANNUITY_RATIO': ['max', 'min', 'mean'],
        'DAYS_FIRST_DUE': ['max', 'min', 'median'],
        'DAYS_DECISION': ['max', 'min', 'median'],
        'HOUR_APPR_PROCESS_START': ['max', 'min', 'median'],
        'AMT_ANNUITY': ['max', 'min', 'median', 'sum'],
        'AMT_CREDIT': ['max', 'min', 'median', 'sum'],
        'AMT_GOODS_PRICE': ['max', 'min', 'median', 'sum'],
        'AMT_APPLICATION': ['max', 'min', 'median', 'sum'],
        'SELLERPLACE_AREA': ['max', 'min', 'median'],
        'DAYS_LAST_DUE_1ST_VERSION': ['max', 'min', 'median'],
}

# Aggregate - joined later to t*_features_df
previous_app_df_avg = new_prev_app_df.groupby('SK_ID_CURR').agg(aggregations)

previous_app_df_avg.head()

PAYMENT_TO_CREDIT_RATIO                      \
                               max       min      mean   
SK_ID_CURR                                               
100001                    0.105940  0.105940  0.105940   
100002                    0.000000  0.000000  0.000000   
100003                    0.101170  0.000000  0.050304   
100004                    0.241719  0.241719  0.241719   
100005                    0.111173  0.049741  0.080457   

           CREDIT_TO_APPLICATION_RATIO                      \
                                   max       min      mean   
SK_ID_CURR                                                   
100001                        0.957782  0.957782  0.957782   
100002                        1.000000  1.000000  1.000000   
100003                        1.150980  0.989013  1.057664   
100004                        0.828021  0.828021  0.828021   
100005                        1.000000  0.899950  0.949975   

           CREDIT_TO_ANNUITY_RATIO                       DAYS_FIRST_DUE  \
                               max        min       mean            max   
SK_ID_CURR                                                                
100001                    6.020501   6.020501   6.020501        -1709.0   
100002                   19.353584  19.353584  19.353584         -565.0   
100003                   10.531859   5.399568   8.677472         -716.0   
100004                    3.753045   3.753045   3.753045         -784.0   
100005                   10.083460   8.342371   9.212916         -706.0   

             ...   AMT_APPLICATION                                  \
             ...               max       min     median        sum   
SK_ID_CURR   ...                                                     
100001       ...           24835.5   24835.5   24835.50    24835.5   
100002       ...          179055.0  179055.0  179055.00   179055.0   
100003       ...          900000.0   68809.5  337500.00  1306309.5   
100004       ...           24282.0   24282.0   24282.00    24282.0   
100005       ...           44617.5       0.0   22308.75    44617.5   

           SELLERPLACE_AREA             DAYS_LAST_DUE_1ST_VERSION          \
                        max  min median                       max     min   
SK_ID_CURR                                                                  
100001                   23   23   23.0                   -1499.0 -1499.0   
100002                  500  500  500.0                     125.0   125.0   
100003                 1400   -1  200.0                    -386.0 -1980.0   
100004                   30   30   30.0                    -694.0  -694.0   
100005                   37   -1   18.0                    -361.0  -376.0   

                    
            median  
SK_ID_CURR          
100001     -1499.0  
100002       125.0  
100003      -647.0  
100004      -694.0  
100005      -368.5  

[5 rows x 40 columns]

In [17]:
# Rename labels
previous_app_df_avg.columns = pd.Index(['PREVIOUS_' + e[0] + "_" + e[1].upper() for e in previous_app_df_avg.columns.tolist()])

previous_app_df_avg.head()

,PREVIOUS_PAYMENT_TO_CREDIT_RATIO_MAX,PREVIOUS_PAYMENT_TO_CREDIT_RATIO_MIN,PREVIOUS_PAYMENT_TO_CREDIT_RATIO_MEAN,PREVIOUS_CREDIT_TO_APPLICATION_RATIO_MAX,PREVIOUS_CREDIT_TO_APPLICATION_RATIO_MIN,PREVIOUS_CREDIT_TO_APPLICATION_RATIO_MEAN,PREVIOUS_CREDIT_TO_ANNUITY_RATIO_MAX,PREVIOUS_CREDIT_TO_ANNUITY_RATIO_MIN,PREVIOUS_CREDIT_TO_ANNUITY_RATIO_MEAN,PREVIOUS_DAYS_FIRST_DUE_MAX,...,PREVIOUS_AMT_APPLICATION_MAX,PREVIOUS_AMT_APPLICATION_MIN,PREVIOUS_AMT_APPLICATION_MEDIAN,PREVIOUS_AMT_APPLICATION_SUM,PREVIOUS_SELLERPLACE_AREA_MAX,PREVIOUS_SELLERPLACE_AREA_MIN,PREVIOUS_SELLERPLACE_AREA_MEDIAN,PREVIOUS_DAYS_LAST_DUE_1ST_VERSION_MAX,PREVIOUS_DAYS_LAST_DUE_1ST_VERSION_MIN,PREVIOUS_DAYS_LAST_DUE_1ST_VERSION_MEDIAN
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,0.105940,0.105940,0.105940,0.957782,0.957782,0.957782,6.020501,6.020501,6.020501,-1709.0,...,24835.5,24835.5,24835.50,24835.5,23,23,23.0,-1499.0,-1499.0,-1499.0
100002,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,19.353584,19.353584,19.353584,-565.0,...,179055.0,179055.0,179055.00,179055.0,500,500,500.0,125.0,125.0,125.0
100003,0.101170,0.000000,0.050304,1.150980,0.989013,1.057664,10.531859,5.399568,8.677472,-716.0,...,900000.0,68809.5,337500.00,1306309.5,1400,-1,200.0,-386.0,-1980.0,-647.0
100004,0.241719,0.241719,0.241719,0.828021,0.828021,0.828021,3.753045,3.753045,3.753045,-784.0,...,24282.0,24282.0,24282.00,24282.0,30,30,30.0,-694.0,-694.0,-694.0
100005,0.111173,0.049741,0.080457,1.000000,0.899950,0.949975,10.083460,8.342371,9.212916,-706.0,...,44617.5,0.0,22308.75,44617.5,37,-1,18.0,-361.0,-376.0,-368.5


In [18]:
# Credit Card

# Load data
credit_card_df = load_credit_data ("credit_card_balance.csv")

# Get counts of number previous applications
credit_card_df_prevs = credit_card_df[['SK_ID_CURR', 'SK_ID_PREV']].groupby('SK_ID_CURR').count()
credit_card_df['SK_ID_PREV'] = credit_card_df['SK_ID_CURR'].map(credit_card_df_prevs['SK_ID_PREV'])

# Create new df 
new_credit_card_df = credit_card_df[['SK_ID_CURR', 'SK_ID_PREV']]

# Add on features
new_credit_card_df['BAL_TO_LIMIT_RATIO'] =  credit_card_df['AMT_BALANCE'] / credit_card_df['AMT_CREDIT_LIMIT_ACTUAL']
new_credit_card_df['PAYMENT_TO_MININSTALL_RATIO'] =  credit_card_df['AMT_PAYMENT_TOTAL_CURRENT'] / credit_card_df['AMT_INST_MIN_REGULARITY'] 

# Drop PREV as would already exist in base file - could leave in if needed
new_credit_card_df = new_credit_card_df.drop ('SK_ID_PREV', axis=1)

# Fill before aggregate
new_credit_card_df.fillna(new_credit_card_df.median(), inplace=True)

# Aggregates
aggregations = {
        'BAL_TO_LIMIT_RATIO': ['max', 'min', 'mean'],
        'PAYMENT_TO_MININSTALL_RATIO': ['max', 'min', 'mean'],
}

# Aggregate - joined later to t*_features_df
credit_card_df_avg = new_credit_card_df.groupby('SK_ID_CURR').agg(aggregations)

credit_card_df_avg.head()

BAL_TO_LIMIT_RATIO                PAYMENT_TO_MININSTALL_RATIO  \
                          max  min      mean                         max   
SK_ID_CURR                                                                 
100006                0.00000  0.0  0.000000                    1.067054   
100011                1.05000  0.0  0.302678                    6.165000   
100013                1.02489  0.0  0.115301                   19.514286   
100021                0.00000  0.0  0.000000                    1.067054   
100023                0.00000  0.0  0.000000                    1.067054   

                                
                 min      mean  
SK_ID_CURR                      
100006      1.067054  1.067054  
100011      0.000000  1.093435  
100013      0.000000  1.688193  
100021      1.067054  1.067054  
100023      1.067054  1.067054

In [19]:
# Rename labels
credit_card_df_avg.columns = pd.Index(['CREDIT_' + e[0] + "_" + e[1].upper() for e in credit_card_df_avg.columns.tolist()])

credit_card_df_avg.head()

,CREDIT_BAL_TO_LIMIT_RATIO_MAX,CREDIT_BAL_TO_LIMIT_RATIO_MIN,CREDIT_BAL_TO_LIMIT_RATIO_MEAN,CREDIT_PAYMENT_TO_MININSTALL_RATIO_MAX,CREDIT_PAYMENT_TO_MININSTALL_RATIO_MIN,CREDIT_PAYMENT_TO_MININSTALL_RATIO_MEAN
SK_ID_CURR,,,,,,
100006,0.00000,0.0,0.000000,1.067054,1.067054,1.067054
100011,1.05000,0.0,0.302678,6.165000,0.000000,1.093435
100013,1.02489,0.0,0.115301,19.514286,0.000000,1.688193
100021,0.00000,0.0,0.000000,1.067054,1.067054,1.067054
100023,0.00000,0.0,0.000000,1.067054,1.067054,1.067054


In [20]:
# Cash Positions

# Load data
pos_cash_df = load_credit_data ("POS_CASH_balance.csv")

# Get counts of number previous applications
pos_cash_df_prevs = pos_cash_df[['SK_ID_CURR', 'SK_ID_PREV']].groupby('SK_ID_CURR').count()
pos_cash_df['SK_ID_PREV'] = pos_cash_df['SK_ID_CURR'].map(pos_cash_df_prevs['SK_ID_PREV'])

# Create new df 
new_pos_cash_df = pos_cash_df[['SK_ID_CURR', 'SK_ID_PREV']]

# Add on features
new_pos_cash_df['FUTURE_TO_TERM_RATIO'] =  pos_cash_df['CNT_INSTALMENT_FUTURE'] / pos_cash_df['CNT_INSTALMENT']

# Get more aggregates for feature important features
new_pos_cash_df['CNT_INSTALMENT_FUTURE'] =  pos_cash_df['CNT_INSTALMENT_FUTURE']

# Drop PREV as would already exist in base file - could leave in if needed
new_pos_cash_df = new_pos_cash_df.drop ('SK_ID_PREV', axis=1)

# Fill before aggregate
new_pos_cash_df.fillna(new_pos_cash_df.median(), inplace=True)

# Aggregates
aggregations = {
        'CNT_INSTALMENT_FUTURE': ['max', 'min', 'median'],
        'FUTURE_TO_TERM_RATIO': ['max', 'min', 'mean'],
}

# Aggregate - joined later to t*_features_df
pos_cash_df_avg = new_pos_cash_df.groupby('SK_ID_CURR').agg(aggregations)

# Rename labels
# pos_cash_df_avg.columns = pd.Index(['CASH_' + e[0] + "_" + e[1].upper() for e in pos_cash_df_avg.columns.tolist()])

pos_cash_df_avg.head()

CNT_INSTALMENT_FUTURE             FUTURE_TO_TERM_RATIO        \
                             max  min median                  max   min   
SK_ID_CURR                                                                
100001                       4.0  0.0    1.0                  1.0  0.00   
100002                      24.0  6.0   15.0                  1.0  0.25   
100003                      12.0  0.0    6.0                  1.0  0.00   
100004                       4.0  0.0    2.5                  1.0  0.00   
100005                      12.0  0.0    7.0                  1.0  0.00   

                      
                mean  
SK_ID_CURR            
100001      0.361111  
100002      0.625000  
100003      0.544643  
100004      0.562500  
100005      0.598485

In [21]:
# Rename labels
pos_cash_df_avg.columns = pd.Index(['CASH_' + e[0] + "_" + e[1].upper() for e in pos_cash_df_avg.columns.tolist()])

pos_cash_df_avg.head()

,CASH_CNT_INSTALMENT_FUTURE_MAX,CASH_CNT_INSTALMENT_FUTURE_MIN,CASH_CNT_INSTALMENT_FUTURE_MEDIAN,CASH_FUTURE_TO_TERM_RATIO_MAX,CASH_FUTURE_TO_TERM_RATIO_MIN,CASH_FUTURE_TO_TERM_RATIO_MEAN
SK_ID_CURR,,,,,,
100001,4.0,0.0,1.0,1.0,0.00,0.361111
100002,24.0,6.0,15.0,1.0,0.25,0.625000
100003,12.0,0.0,6.0,1.0,0.00,0.544643
100004,4.0,0.0,2.5,1.0,0.00,0.562500
100005,12.0,0.0,7.0,1.0,0.00,0.598485


In [22]:
# Installments

# Load data
installments_df = load_credit_data ("installments_payments.csv")

# Get counts of number previous applications
installments_df_prevs = installments_df[['SK_ID_CURR', 'SK_ID_PREV']].groupby('SK_ID_CURR').count()
installments_df['SK_ID_PREV'] = installments_df['SK_ID_CURR'].map(installments_df_prevs['SK_ID_PREV'])

# Create new df 
new_installments_df = installments_df[['SK_ID_CURR', 'SK_ID_PREV']]

# Add on features
new_installments_df['INSTPAY_PAY_TO_INSTALL_RATIO'] =  installments_df['AMT_PAYMENT'] / installments_df['AMT_INSTALMENT']
new_installments_df['INSTPAY_DIFF_DAYS_INSTALL_DAYS_PAY'] =  installments_df['DAYS_INSTALMENT'] - installments_df['DAYS_ENTRY_PAYMENT']

# Get more aggregates for feature important features
new_installments_df['AMT_PAYMENT'] =  installments_df['AMT_PAYMENT']
new_installments_df['AMT_INSTALMENT'] =  installments_df['AMT_INSTALMENT']
new_installments_df['DAYS_ENTRY_PAYMENT'] =  installments_df['DAYS_ENTRY_PAYMENT']
new_installments_df['DAYS_INSTALMENT'] =  installments_df['DAYS_INSTALMENT']

# Drop PREV as would already exist in base file - could leave in if needed
new_installments_df = new_installments_df.drop ('SK_ID_PREV', axis=1)

# Fill before aggregate
new_installments_df.fillna(new_installments_df.median(), inplace=True)

# Aggregates
aggregations = {
        'AMT_PAYMENT': ['max', 'min', 'median', 'sum'],
        'AMT_INSTALMENT': ['max', 'min', 'median', 'sum'],
        'DAYS_ENTRY_PAYMENT': ['max', 'min', 'median'],
        'DAYS_INSTALMENT': ['max', 'min', 'median'],
        'INSTPAY_PAY_TO_INSTALL_RATIO': ['max', 'min','mean'],
        'INSTPAY_DIFF_DAYS_INSTALL_DAYS_PAY': ['max', 'min','mean'],
}

# Aggregate - joined later to t*_features_df
installments_df_avg = new_installments_df.groupby('SK_ID_CURR').agg(aggregations)

installments_df_avg.head()

AMT_PAYMENT                                   AMT_INSTALMENT  \
                   max       min     median          sum            max   
SK_ID_CURR                                                                
100001       17397.900  3951.000   3980.925    41195.925      17397.900   
100002       53093.745  9251.775   9251.775   219625.695      53093.745   
100003      560835.360  6662.970  64275.615  1618864.650     560835.360   
100004       10573.965  5357.250   5357.250    21288.465      10573.965   
100005       17656.245  4813.200   4813.200    56161.845      17656.245   

                                             DAYS_ENTRY_PAYMENT          \
                 min     median          sum                max     min   
SK_ID_CURR                                                                
100001      3951.000   3980.925    41195.925            -1628.0 -2916.0   
100002      9251.775   9251.775   219625.695              -49.0  -587.0   
100003      6662.970  64275.615  1618864.650             -544.0 -2324.0   
100004      5357.250   5357.250    21288.465             -727.0  -795.0   
100005      4813.200   4813.200    56161.845             -470.0  -736.0   

                   DAYS_INSTALMENT                  \
            median             max     min  median   
SK_ID_CURR                                           
100001     -1715.0         -1619.0 -2916.0 -1709.0   
100002      -312.0           -25.0  -565.0  -295.0   
100003      -806.0          -536.0 -2310.0  -797.0   
100004      -763.0          -724.0  -784.0  -754.0   
100005      -585.0          -466.0  -706.0  -586.0   

           INSTPAY_PAY_TO_INSTALL_RATIO            \
                                    max  min mean   
SK_ID_CURR                                          
100001                              1.0  1.0  1.0   
100002                              1.0  1.0  1.0   
100003                              1.0  1.0  1.0   
100004                              1.0  1.0  1.0   
100005                              1.0  1.0  1.0   

           INSTPAY_DIFF_DAYS_INSTALL_DAYS_PAY                   
                                          max   min       mean  
SK_ID_CURR                                                      
100001                                   36.0 -11.0   7.285714  
100002                                   31.0  12.0  20.421053  
100003                                   14.0   1.0   7.160000  
100004                                   11.0   3.0   7.666667  
100005                                   37.0  -1.0  23.555556

In [23]:
# Rename labels
installments_df_avg.columns = pd.Index(['INSTALL_' + e[0] + "_" + e[1].upper() for e in installments_df_avg.columns.tolist()])

installments_df_avg.head()

,INSTALL_AMT_PAYMENT_MAX,INSTALL_AMT_PAYMENT_MIN,INSTALL_AMT_PAYMENT_MEDIAN,INSTALL_AMT_PAYMENT_SUM,INSTALL_AMT_INSTALMENT_MAX,INSTALL_AMT_INSTALMENT_MIN,INSTALL_AMT_INSTALMENT_MEDIAN,INSTALL_AMT_INSTALMENT_SUM,INSTALL_DAYS_ENTRY_PAYMENT_MAX,INSTALL_DAYS_ENTRY_PAYMENT_MIN,INSTALL_DAYS_ENTRY_PAYMENT_MEDIAN,INSTALL_DAYS_INSTALMENT_MAX,INSTALL_DAYS_INSTALMENT_MIN,INSTALL_DAYS_INSTALMENT_MEDIAN,INSTALL_INSTPAY_PAY_TO_INSTALL_RATIO_MAX,INSTALL_INSTPAY_PAY_TO_INSTALL_RATIO_MIN,INSTALL_INSTPAY_PAY_TO_INSTALL_RATIO_MEAN,INSTALL_INSTPAY_DIFF_DAYS_INSTALL_DAYS_PAY_MAX,INSTALL_INSTPAY_DIFF_DAYS_INSTALL_DAYS_PAY_MIN,INSTALL_INSTPAY_DIFF_DAYS_INSTALL_DAYS_PAY_MEAN
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,
100001,17397.900,3951.000,3980.925,41195.925,17397.900,3951.000,3980.925,41195.925,-1628.0,-2916.0,-1715.0,-1619.0,-2916.0,-1709.0,1.0,1.0,1.0,36.0,-11.0,7.285714
100002,53093.745,9251.775,9251.775,219625.695,53093.745,9251.775,9251.775,219625.695,-49.0,-587.0,-312.0,-25.0,-565.0,-295.0,1.0,1.0,1.0,31.0,12.0,20.421053
100003,560835.360,6662.970,64275.615,1618864.650,560835.360,6662.970,64275.615,1618864.650,-544.0,-2324.0,-806.0,-536.0,-2310.0,-797.0,1.0,1.0,1.0,14.0,1.0,7.160000
100004,10573.965,5357.250,5357.250,21288.465,10573.965,5357.250,5357.250,21288.465,-727.0,-795.0,-763.0,-724.0,-784.0,-754.0,1.0,1.0,1.0,11.0,3.0,7.666667
100005,17656.245,4813.200,4813.200,56161.845,17656.245,4813.200,4813.200,56161.845,-470.0,-736.0,-585.0,-466.0,-706.0,-586.0,1.0,1.0,1.0,37.0,-1.0,23.555556


In [24]:
 # print (new_installments_df_avg ['INSTPAY_INSTALL_TO_PAY_RATIO'] != 1)  # ---> A few True

In [29]:
# Merging - training
training_df_merged = tr_features_df
training_df_merged = training_df_merged.merge(right=previous_app_df_avg.reset_index(), on = 'SK_ID_CURR', how = 'left')
training_df_merged = training_df_merged.merge(right=credit_card_df_avg.reset_index(), on = 'SK_ID_CURR', how = 'left')
training_df_merged = training_df_merged.merge(right=pos_cash_df_avg.reset_index(), on = 'SK_ID_CURR', how = 'left')
training_df_merged = training_df_merged.merge(right=installments_df_avg.reset_index(), on = 'SK_ID_CURR', how = 'left')

# In case any of the math above resulted in an inf, i.e. division of 0.
training_df_merged = training_df_merged.replace([np.inf, -np.inf], np.nan)

# Deal with NULLs from the merges - if no info exists, just assume 0.
training_df_merged.fillna(0, inplace=True)

training_df_merged.head()

,SK_ID_CURR,APP_CREDIT_TO_ANNUITY_RATIO,APP_CREDIT_TO_GOODS_RATIO,APP_INCOME_PER_CHLD,APP_CREDIT_TO_INCOME_RATIO,APP_ANNUITY_TO_INCOME_RATIO,APP_SOURCES_PROD,APP_EXT_SOURCES_MEAN,APP_EXT_SCORES_STD,APP_CAR_TO_BIRTH_RATIO,...,INSTALL_DAYS_ENTRY_PAYMENT_MEDIAN,INSTALL_DAYS_INSTALMENT_MAX,INSTALL_DAYS_INSTALMENT_MIN,INSTALL_DAYS_INSTALMENT_MEDIAN,INSTALL_INSTPAY_PAY_TO_INSTALL_RATIO_MAX,INSTALL_INSTPAY_PAY_TO_INSTALL_RATIO_MIN,INSTALL_INSTPAY_PAY_TO_INSTALL_RATIO_MEAN,INSTALL_INSTPAY_DIFF_DAYS_INSTALL_DAYS_PAY_MAX,INSTALL_INSTPAY_DIFF_DAYS_INSTALL_DAYS_PAY_MIN,INSTALL_INSTPAY_DIFF_DAYS_INSTALL_DAYS_PAY_MEAN
0,100002,16.461104,1.158397,202500.0,2.007889,0.121977,0.003043,0.161787,0.092026,-0.000627,...,-312.0,-25.0,-565.0,-295.0,1.0,1.00000,1.000000,31.0,12.0,20.421053
1,100003,36.234085,1.145199,270000.0,4.790750,0.132216,0.119932,0.466757,0.219895,-0.000627,...,-806.0,-536.0,-2310.0,-797.0,1.0,1.00000,1.000000,14.0,1.0,7.160000
2,100004,20.000000,1.000000,67500.0,2.000000,0.099999,0.119932,0.642739,0.122792,-0.001365,...,-763.0,-724.0,-784.0,-754.0,1.0,1.00000,1.000000,11.0,3.0,7.666667
3,100006,10.532818,1.052803,135000.0,2.316167,0.219898,0.119932,0.650442,0.136021,-0.000627,...,-211.0,-11.0,-545.0,-206.0,1.0,1.00000,1.000000,77.0,1.0,19.375000
4,100007,23.461618,1.000000,121500.0,4.222222,0.179961,0.119932,0.322738,0.136021,-0.000627,...,-852.5,-14.0,-2326.0,-851.0,1.0,0.00005,0.954545,31.0,-12.0,3.636364


In [30]:
# Merging - training
testing_df_merged = te_features_df
testing_df_merged = testing_df_merged.merge(right=previous_app_df_avg.reset_index(), on = 'SK_ID_CURR', how = 'left')
testing_df_merged = testing_df_merged.merge(right=credit_card_df_avg.reset_index(), on = 'SK_ID_CURR', how = 'left')
testing_df_merged = testing_df_merged.merge(right=pos_cash_df_avg.reset_index(), on = 'SK_ID_CURR', how = 'left')
testing_df_merged = testing_df_merged.merge(right=installments_df_avg.reset_index(), on = 'SK_ID_CURR', how = 'left')

# In case any of the math above resulted in an inf, i.e. division of 0.
testing_df_merged = testing_df_merged.replace([np.inf, -np.inf], np.nan)

# Deal with NULLs from the merges - if no info exists, just assume 0.
testing_df_merged.fillna(0, inplace=True)

testing_df_merged.head()

,SK_ID_CURR,APP_CREDIT_TO_ANNUITY_RATIO,APP_CREDIT_TO_GOODS_RATIO,APP_INCOME_PER_CHLD,APP_CREDIT_TO_INCOME_RATIO,APP_ANNUITY_TO_INCOME_RATIO,APP_SOURCES_PROD,APP_EXT_SOURCES_MEAN,APP_EXT_SCORES_STD,APP_CAR_TO_BIRTH_RATIO,...,INSTALL_DAYS_ENTRY_PAYMENT_MEDIAN,INSTALL_DAYS_INSTALMENT_MAX,INSTALL_DAYS_INSTALMENT_MIN,INSTALL_DAYS_INSTALMENT_MEDIAN,INSTALL_INSTPAY_PAY_TO_INSTALL_RATIO_MAX,INSTALL_INSTPAY_PAY_TO_INSTALL_RATIO_MIN,INSTALL_INSTPAY_PAY_TO_INSTALL_RATIO_MEAN,INSTALL_INSTPAY_DIFF_DAYS_INSTALL_DAYS_PAY_MAX,INSTALL_INSTPAY_DIFF_DAYS_INSTALL_DAYS_PAY_MIN,INSTALL_INSTPAY_DIFF_DAYS_INSTALL_DAYS_PAY_MEAN
0,100001,27.664697,1.2640,135000.0,4.213333,0.152299,0.094803,0.567263,0.353601,-0.000624,...,-1715.0,-1619.0,-2916.0,-1709.0,1.0,1.000000,1.000000,36.0,-11.0,7.285714
1,100005,12.824870,1.2376,99000.0,2.250182,0.175453,0.071345,0.429869,0.136694,-0.000624,...,-585.0,-466.0,-706.0,-586.0,1.0,1.000000,1.000000,37.0,-1.0,23.555556
2,100013,9.505482,1.0528,202500.0,3.275378,0.344576,0.119686,0.655389,0.062788,-0.000250,...,-1383.0,-14.0,-2705.0,-1383.0,1.0,0.000266,0.935484,38.0,-21.0,5.180645
3,100028,32.130726,1.0000,105000.0,5.000000,0.155614,0.164177,0.549372,0.055432,-0.000624,...,-812.0,-27.0,-1773.0,-812.0,1.0,0.030496,0.911504,19.0,-7.0,3.000000
4,100038,19.506034,1.0000,90000.0,3.475000,0.178149,0.119686,0.313916,0.158068,-0.001227,...,-634.0,-457.0,-787.0,-622.0,1.0,1.000000,1.000000,18.0,9.0,12.250000


In [31]:
training_df_merged.describe()

,SK_ID_CURR,APP_CREDIT_TO_ANNUITY_RATIO,APP_CREDIT_TO_GOODS_RATIO,APP_INCOME_PER_CHLD,APP_CREDIT_TO_INCOME_RATIO,APP_ANNUITY_TO_INCOME_RATIO,APP_SOURCES_PROD,APP_EXT_SOURCES_MEAN,APP_EXT_SCORES_STD,APP_CAR_TO_BIRTH_RATIO,...,INSTALL_DAYS_ENTRY_PAYMENT_MEDIAN,INSTALL_DAYS_INSTALMENT_MAX,INSTALL_DAYS_INSTALMENT_MIN,INSTALL_DAYS_INSTALMENT_MEDIAN,INSTALL_INSTPAY_PAY_TO_INSTALL_RATIO_MAX,INSTALL_INSTPAY_PAY_TO_INSTALL_RATIO_MIN,INSTALL_INSTPAY_PAY_TO_INSTALL_RATIO_MEAN,INSTALL_INSTPAY_DIFF_DAYS_INSTALL_DAYS_PAY_MAX,INSTALL_INSTPAY_DIFF_DAYS_INSTALL_DAYS_PAY_MIN,INSTALL_INSTPAY_DIFF_DAYS_INSTALL_DAYS_PAY_MEAN
count,307511.000000,307511.000000,307511.000000,3.075110e+05,307511.000000,307511.000000,3.075110e+05,307511.000000,3.075110e+05,307511.000000,...,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000
mean,278180.518577,21.612259,1.122991,1.395079e+05,3.957570,0.180928,1.282654e-01,0.509260,1.494183e-01,-0.000704,...,-846.410732,-303.141546,-1503.163536,-835.695445,12.936899,0.565147,1.311532,35.895324,-15.393397,10.623410
std,102790.175348,7.823677,0.123989,1.458110e+05,2.689728,0.094571,6.520143e-02,0.149761,9.385374e-02,0.000566,...,673.765801,530.278613,956.409762,674.311019,864.272268,0.478381,29.391051,29.428250,108.250100,13.041179
min,100002.000000,8.036674,0.150000,3.000000e+03,0.004808,0.000224,2.430332e-07,0.000006,3.538459e-07,-0.009197,...,-3071.000000,-2922.000000,-2922.000000,-2922.000000,0.000000,0.000000,0.000000,-42.000000,-2884.000000,-1884.204545
25%,189145.500000,15.614555,1.000000,7.875000e+04,2.018667,0.114784,1.199324e-01,0.413716,8.101753e-02,-0.000627,...,-1265.000000,-348.000000,-2436.000000,-1254.000000,1.000000,0.007440,0.937500,20.000000,-8.000000,5.250000
50%,278202.000000,20.000000,1.118800,1.170000e+05,3.265067,0.162832,1.199324e-01,0.524502,1.360209e-01,-0.000627,...,-636.000000,-35.000000,-1450.000000,-624.000000,1.000000,1.000000,1.000000,31.000000,-1.000000,9.108696
75%,367142.500000,27.099985,1.198000,1.800000e+05,5.159880,0.229061,1.199324e-01,0.622757,2.016881e-01,-0.000627,...,-323.000000,-16.000000,-621.000000,-310.000000,1.000000,1.000000,1.000000,42.000000,2.000000,14.423077
max,456255.000000,45.305079,6.000000,5.850000e+07,84.736842,1.875892,6.185571e-01,0.878903,6.519439e-01,-0.000000,...,0.000000,0.000000,0.000000,0.000000,194250.000000,1.600495,8482.446155,3189.000000,156.000000,295.000000


In [32]:
testing_df_merged.describe()

,SK_ID_CURR,APP_CREDIT_TO_ANNUITY_RATIO,APP_CREDIT_TO_GOODS_RATIO,APP_INCOME_PER_CHLD,APP_CREDIT_TO_INCOME_RATIO,APP_ANNUITY_TO_INCOME_RATIO,APP_SOURCES_PROD,APP_EXT_SOURCES_MEAN,APP_EXT_SCORES_STD,APP_CAR_TO_BIRTH_RATIO,...,INSTALL_DAYS_ENTRY_PAYMENT_MEDIAN,INSTALL_DAYS_INSTALMENT_MAX,INSTALL_DAYS_INSTALMENT_MIN,INSTALL_DAYS_INSTALMENT_MEDIAN,INSTALL_INSTPAY_PAY_TO_INSTALL_RATIO_MAX,INSTALL_INSTPAY_PAY_TO_INSTALL_RATIO_MIN,INSTALL_INSTPAY_PAY_TO_INSTALL_RATIO_MEAN,INSTALL_INSTPAY_DIFF_DAYS_INSTALL_DAYS_PAY_MAX,INSTALL_INSTPAY_DIFF_DAYS_INSTALL_DAYS_PAY_MIN,INSTALL_INSTPAY_DIFF_DAYS_INSTALL_DAYS_PAY_MEAN
count,48744.000000,48744.000000,48744.000000,4.874400e+04,48744.000000,48744.000000,4.874400e+04,48744.000000,48744.000000,48744.000000,...,48744.000000,48744.000000,48744.000000,48744.000000,48744.000000,48744.000000,48744.000000,48744.000000,48744.000000,48744.000000
mean,277796.676350,17.173426,1.131888,1.486187e+05,3.167544,0.182974,1.293088e-01,0.506972,0.148615,-0.000694,...,-879.752503,-318.384601,-1586.586247,-868.139258,11.006952,0.595091,1.202508,37.888786,-12.229526,11.407424
std,103169.547296,6.290623,0.124697,9.765253e+04,2.137173,0.095455,7.116662e-02,0.139813,0.089111,0.000529,...,663.182780,539.062872,937.734097,663.935856,446.814174,0.471902,13.469118,28.399428,92.131389,11.828743
min,100001.000000,8.036674,0.650000,7.312500e+03,0.092500,0.005730,2.707109e-08,0.000557,0.000008,-0.008037,...,-2922.000000,-2922.000000,-2922.000000,-2922.000000,0.000000,0.000000,0.000000,-13.000000,-2651.000000,-688.500000
25%,188557.750000,12.587749,1.000000,9.000000e+04,1.695695,0.116395,1.196864e-01,0.417207,0.082703,-0.000624,...,-1265.000000,-382.000000,-2550.000000,-1255.000000,1.000000,0.015358,0.959016,22.000000,-7.000000,5.900573
50%,277549.000000,16.277105,1.132000,1.350000e+05,2.666143,0.165921,1.196864e-01,0.519572,0.138058,-0.000624,...,-671.000000,-38.000000,-1538.000000,-658.500000,1.000000,1.000000,1.000000,31.000000,-1.000000,9.565217
75%,367555.500000,20.600751,1.211200,1.800000e+05,4.029600,0.229798,1.196864e-01,0.610495,0.201322,-0.000624,...,-371.000000,-16.000000,-700.000000,-357.000000,1.000000,1.000000,1.000000,44.000000,2.000000,14.872596
max,456250.000000,32.219264,4.000000,4.410000e+06,30.445000,2.024650,5.484052e-01,0.855000,0.619230,-0.000000,...,0.000000,0.000000,0.000000,0.000000,68314.500000,1.969512,2733.540001,769.000000,30.000000,265.047619


In [33]:
# Saving to files ... to be added to appropriate base file later
training_df_merged.to_csv('training_new_features_v1.csv', index = False)
testing_df_merged.to_csv('testing_new_features_v1.csv', index = False)